## **First we apply our method for feat. ectraction, then only XGB hen XGB with Optuna**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner

/content/drive/.shortcut-targets-by-id/1PND44K5yRW8mjP-NB4mfwEivBcL8QvfE/2021_VIIT08_P300/Dataset/bci-ner


In [ ]:
!pip install -U imbalanced-learn
!pip install pyriemann

Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.7/dist-packages (0.8.0)


In [ ]:
import pandas as pd
import random
from pyriemann.estimation import XdawnCovariances , Covariances
from pyriemann.tangentspace import TangentSpace
from os.path import isfile
import numpy as np
import h5py
from imblearn.over_sampling import RandomOverSampler

In [ ]:
X_train= np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/Proposed Feature Extraction using mne/X_train.npy')
Y_train=np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/Y_train.npy')
Y_test = np.reshape(pd.read_csv('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/true_labels.csv', header=None).values, 3400)
X_test= np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/Proposed Feature Extraction using mne/X_test.npy')

In [ ]:
np.count_nonzero(Y_train == 0),np.count_nonzero(Y_train == 1)

(1590, 3850)

In [ ]:
pip install numpy matplotlib scipy numba scikit-learn mne PyWavelets pandas

In [ ]:
pip install mne-features

In [ ]:
from mne_features.feature_extraction import extract_features
from mne_features.univariate import get_univariate_funcs

In [ ]:

X_train.shape
X_train=np.reshape(X_train,(16*340, 56,140))

ValueError: ignored

In [ ]:
X_train.shape

In [ ]:
params1=['mean','variance','std']
new_X1=extract_features(X_train,128, params1 )
new_X1.shape

In [ ]:
params2=['higuchi_fd','hurst_exp']
new_X2=extract_features(X_train,128, params2 )
new_X2.shape

In [ ]:

params3=['spect_entropy','ptp_amp']
new_X3=extract_features(X_train,128, params3 )
new_X3.shape

In [ ]:
params4 =['wavelet_coef_energy']
new_x4 =extract_features(X_train,128, params4 )
new_x4.shape

In [ ]:
XC = XdawnCovariances(nfilter=4,estimator='oas',xdawn_estimator="oas") # our transformer
TS = TangentSpace(metric='riemann')

X_train_newX = XC.fit_transform(X_train,Y_train)
# X_train = fit_transform(train_data)
print("Xdawn Covariances", X_train_newX.shape)

In [ ]:
X_train_newX = TS.fit_transform(X_train_newX)
print("Tangent Spaces", X_train_newX.shape)

In [ ]:
boss= np.concatenate((new_X1,new_X3,new_X2,new_x4,X_train_newX),axis=1)


In [ ]:
boss.shape

In [ ]:
## Doing the same with X_test

In [ ]:
X_test.shape
X_test=X_test.reshape(10*340, 56,140)

In [ ]:
params1=['mean','variance','std']
new_X1=extract_features(X_test,128, params1 )
new_X1.shape

In [ ]:
params2=['higuchi_fd','hurst_exp']
new_X2=extract_features(X_test,128, params2 )
new_X2.shape

In [ ]:
params3=['spect_entropy','ptp_amp']
new_X3=extract_features(X_test,128, params3 )
new_X3.shape

In [ ]:
params4 =['wavelet_coef_energy']
new_x4 =extract_features(X_test,128, params4 )
new_x4.shape

In [ ]:
X_test = XC.transform(X_test)
# X_test = Covariances('oas').fit_transform(X_test)
X_test = TS.transform(X_test)


In [ ]:
X_test_new= np.concatenate((new_X1,new_X3,new_X2,new_x4, X_test),axis=1)


# Applying only XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.utils import class_weight
from sklearn import metrics
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix


In [ ]:
Y_train.shape, Y_test.shape, X_train.shape, X_test.shape

((5440,), (3400,), (5440, 752), (3400, 752))

In [ ]:
primary_eval_metric = metrics.f1_score
confusion_lbs = ['neg', 'pos']

## Compute `class_weights` using sklearn
cls_weight = (Y_train.shape[0] - np.sum(Y_train)) / np.sum(Y_train)
# fit model no training data
model = XGBClassifier(scale_pos_weight=cls_weight)
model.fit(X_train, Y_train)
y_preds=model.predict(X_test)
cls = classification_report(Y_test,y_preds)
print(cls)
print("------------COnfusion Matrix---------------")
confusion_matrix(Y_test,y_preds)


              precision    recall  f1-score   support

           0       0.35      0.37      0.36       989
           1       0.74      0.72      0.73      2411

    accuracy                           0.62      3400
   macro avg       0.54      0.54      0.54      3400
weighted avg       0.62      0.62      0.62      3400

------------COnfusion Matrix---------------


array([[ 364,  625],
       [ 677, 1734]])

In [ ]:
!pip install optuna

### Hyperparameter Tuning Usinng Optuna + Plots

In [ ]:
import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score


In [ ]:
def objective(trial: Trial,X,y) -> float:
    
    #joblib.dump(study, 'study.pkl')
    
    

    param = {
                "n_estimators" : trial.suggest_int('n_estimators', 0, 1000),
                'max_depth':trial.suggest_int('max_depth', 2, 25),
                'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
                'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
                #'min_child_weight':trial.suggest_int('min_child_weight', 0, 5),
                'gamma':trial.suggest_int('gamma', 0, 5),
                'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
                'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
                'nthread' : -1,
                #'tree_method' : "gpu_hist", 

                
            }
    model = XGBClassifier(**param)

    model.fit(X_train,Y_train)

    return cross_val_score(model,X_test, Y_test).mean()

In [ ]:
study = optuna.create_study(direction='maximize',sampler=TPESampler())
study.optimize(lambda trial : objective(trial,X_train,Y_train),n_trials= 10)

[I 2021-06-08 08:15:23,679] A new study created in memory with name: no-name-073729c0-1de0-4c77-a272-6b5213ab7c22
[I 2021-06-08 08:52:47,657] Trial 0 finished with value: 0.5891176470588235 and parameters: {'n_estimators': 873, 'max_depth': 11, 'reg_alpha': 5, 'reg_lambda': 2, 'gamma': 3, 'learning_rate': 0.007514773013450848, 'colsample_bytree': 0.78}. Best is trial 0 with value: 0.5891176470588235.
[I 2021-06-08 08:55:03,723] Trial 1 finished with value: 0.5882352941176471 and parameters: {'n_estimators': 61, 'max_depth': 23, 'reg_alpha': 1, 'reg_lambda': 2, 'gamma': 3, 'learning_rate': 0.3859512178815996, 'colsample_bytree': 0.56}. Best is trial 0 with value: 0.5891176470588235.
[I 2021-06-08 09:01:25,564] Trial 2 finished with value: 0.5914705882352942 and parameters: {'n_estimators': 288, 'max_depth': 22, 'reg_alpha': 4, 'reg_lambda': 5, 'gamma': 2, 'learning_rate': 0.2893601367013301, 'colsample_bytree': 0.31}. Best is trial 2 with value: 0.5914705882352942.


In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
######## reducing nestimators

In [ ]:
def objective(trial: Trial,X,y) -> float:
    
    #joblib.dump(study, 'study.pkl')
    
    

    param = {
                "n_estimators" : trial.suggest_int('n_estimators', 0, 100),
                'max_depth':trial.suggest_int('max_depth', 2, 25),
                'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
                'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
                #'min_child_weight':trial.suggest_int('min_child_weight', 0, 5),
                #'gamma':trial.suggest_int('gamma', 0, 5),
                'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
                'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
                'nthread' : -1,
                'tree_method' : "gpu_hist", 

                
            }
    model = XGBClassifier(**param)

    model.fit(boss,Y_train)

    return cross_val_score(model,X_test_new, Y_test).mean()

In [ ]:
study = optuna.create_study(direction='maximize',sampler=TPESampler())
study.optimize(lambda trial : objective(trial,boss,Y_train),
               n_trials= 10)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial
print

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
y_preds=lgbm.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
cls = classification_report(y_test,y_preds)
print(cls)
print("------------COnfusion Matrix---------------")
confusion_matrix(y_test,y_preds)

### Hyperparameter using GA

## Hyperparamet using Grid and Random